In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import toolz as tz

In [3]:
import pandas
pandas.set_option('display.max_rows', None)
pandas.set_option('display.width', None)

## Warehousing Financing

### Deal Strcuture

For origniator who seeks external funding to borrower . `Warehousing` is one of source to faciliate the operation which is provided by a `Liquidity Facility`.

The `Liquidity Facility` usually :
* provide a `line of credit` as maximum balance
* it charges interest over the balance drawed
* it charges fee over the balance undrawed

The `Originator` would use the funding to buy assets and plug it into to pool:
* the cashflow from the pool maybe used to purchase extra asset or repay the `line of credit`

`Call` is executed with certain condition met
* maybe a certain `date` has passed
* a pool balance reached
* etc

Once the deal was called, then all proceeds from selling assets will repay all the liablities of the deal

### Model the `Call`

To model `call` there are two components:

#### When to `call` ? 

`absbox` can use `<Condition>` to model when the call can be triggered, which is passed into `run(runAssump=[])`.

    ("call", ("if", <Condition>))


The `<Condition>` can be test on a date; test on `poolBalance`  or test on certain `<Formula>`, or `periods` of the deal.

In warehousing context, it ususally hit the end if `poolBalance` reach to a certain level.

In [4]:
callByDate = ("call",("if",["date",">","2022-09-20"]))

#### What happed of the `call` ?

User can define a sequence of waterfall actions to be performed. 

In warehousing context, it ususally will
* sell all pool cash with a `<pricing method>`
* pay off expenses if any
* pay off the liquidity facility's due interest, fees and balance
* pay residual to the liquidity facility as yield, if any

In [5]:
cleanUpActions = \
    {"cleanUp":[
        ["sellAsset", ["PvRate", 0.08], "acc01"]
        ,["liqRepay",["int","fee","bal"], "acc01", "warehouse"]
        ,["liqRepayResidual", "acc01", "warehouse"]
    ]}

### Model the `Liquidity Facility`

* `credit`, which means available balance to draw
    * it could be a fix amount of the life time
    * or it can be bound to a ``<formula>``
    * or it just ``unlimited`` 
* `balance`, balance drawn
* `rateType`, interest calcualtion on balance drawn
* `feeType`, fee calcuation on balanc un-drawned

In [42]:
from absbox import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2025-01-01"}
    ,{'assets':[["Mortgage" ,{"originBalance":0,"originRate":["fix",0.045],"originTerm":30
                            ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                    ,{"currentBalance":0 ,"currentRate":0.08 ,"remainTerm":24 ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),("buyAcc",None))
    ,tuple()
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{("DistributionDay","Warehousing"):[
         ["payFee","acc01",['trusteeFee']]
         ,["liqAccrue","warehouse"]
         ,["liqSupport", "warehouse", "account", "buyAcc", {"formula":("custom","buyCurve")}]
         ,["buyAsset",["Current|Defaulted",0.3,0],"buyAcc",None]
         ,["liqRepay",["int","fee","bal"],"acc01","warehouse"],
        ]
    }|cleanUpActions
    ,[["CollectedCash","acc01"]]
    ,{"warehouse":{
        "credit" : 2500
        ,"balance": 1500
        ,"type": {"total": 4000}
        ,"creditCalc":"IncludeDueInt"
        ,"start": "2021-06-15"
        ,"rateType": ("floater",0.05,{"index":"SOFR1Y","spread":-0.0169,"reset":"MonthEnd"}) # begin rate
        ,"feeType": ("fix", 0.03)
        ,"rate":0.03 # current rate
        ,"fee":0.01
        }
     }
    ,None
    ,None
    ,None
    ,"Warehousing"
    ,{"buyCurve": {"BalanceCurve": [
        ["2022-01-01",500]
        ,["2022-06-01",1000]
        ,["2022-08-01",1000]
        ,["2022-10-01",1000]
    ] 
    }
     })

In [71]:
from absbox import API,EnginePath,readInspect
localAPI = API(EnginePath.DEV,check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.30.4

In [72]:
revol_asset = ["Mortgage"
                ,{"originBalance":10,"originRate":["fix",0.045],"originTerm":30
                  ,"freq":"Monthly","type":"Level","originDate":"2021-07-01"}
                  ,{"currentBalance":10
                  ,"currentRate":0.08
                  ,"remainTerm":12
                  ,"status":"current"}]


r = localAPI.run(test01,
           runAssump=[("revolving"
                         ,["constant",revol_asset]
                         ,("Pool",("Mortgage",None,None,None,None)
                                   ,None
                                   ,None))
                      ,("interest",("SOFR1Y",0.08))
                      ,("inspect",("MonthEnd",[("poolBalance",),("liqBalance","warehouse"),("liqCredit","warehouse")]))
                      ,callByDate
                      #,("stop","2022-05-21")
                       ],
           read=True)

Warning Message from server:

In [75]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-01-20,1666.66,0.00,0.00,0,0,0,0,0.08,None,None,0.00,0,0,0,0,0
2022-02-20,3199.45,133.87,11.10,0,0,0,0,0.08,None,None,133.87,0,0,0,0,0
2022-03-20,4597.48,268.63,21.31,0,0,0,0,0.08,None,None,402.50,0,0,0,0,0
2022-04-20,5828.62,404.29,30.62,0,0,0,0,0.08,None,None,806.79,0,0,0,0,0
2022-05-20,5290.28,538.34,38.83,0,0,0,0,0.08,None,None,1345.13,0,0,0,0,0
2022-06-20,4754.77,541.94,35.24,0,0,0,0,0.08,None,None,1887.07,0,0,0,0,0
2022-07-20,4219.41,546.06,31.67,0,0,0,0,0.08,None,None,2433.13,0,0,0,0,0
2022-08-20,3678.44,550.57,28.09,0,0,0,0,0.08,None,None,2983.70,0,0,0,0,0
2022-09-20,3130.56,555.01,24.48,0,0,0,0,0.08,None,None,3538.71,0,0,0,0,0


In [74]:
r['liqProvider']['warehouse']

,credit,used,int,premium,cash,memo
date,,,,,,
2021-06-15,2000.00,300.00,0.00,0.00,0.00,
2021-07-26,1997.88,300.00,2.12,6.73,0.00,"<SupportExp:(Int:2.12,Fee:6.73)>"
2021-07-26,1997.88,300.00,2.12,6.73,0.00,<Draw:>
2021-07-26,1997.88,300.00,2.12,6.73,0.00,<Repay:LiqInt>
2021-07-26,1997.88,300.00,2.12,6.73,0.00,<Repay:LiqPremium>
2021-07-26,1997.88,300.00,2.12,6.73,0.00,<Repay:LiqBal>
2021-08-20,1996.59,300.00,3.41,10.83,0.00,"<SupportExp:(Int:1.29,Fee:4.10)>"
2021-08-20,1996.59,300.00,3.41,10.83,0.00,<Draw:>
2021-08-20,1996.59,300.00,3.41,10.83,0.00,<Repay:LiqInt>


In [66]:
500.00 * 0.03 * (30/365)

1.2328767123287672

In [46]:
r['liqProvider']['warehouse']

,credit,used,int,premium,cash,memo
date,,,,,,
2021-06-15,2000.00,300.00,0.00,0.00,0.00,
2021-07-26,1999.06,300.00,0.94,6.73,0.00,"<SupportExp:(Int:0.94,Fee:6.73)>"
2021-07-26,1999.06,300.00,0.94,6.73,0.00,<Draw:>
2021-07-26,1999.06,300.00,0.94,6.73,0.00,<Repay:LiqInt>
2021-07-26,1999.06,300.00,0.94,6.73,0.00,<Repay:LiqPremium>
2021-07-26,1999.06,300.00,0.94,6.73,0.00,<Repay:LiqBal>
2021-08-20,1997.54,300.00,2.46,17.57,0.00,"<SupportExp:(Int:1.52,Fee:10.84)>"
2021-08-20,1997.54,300.00,2.46,17.57,0.00,<Draw:>
2021-08-20,1997.54,300.00,2.46,17.57,0.00,<Repay:LiqInt>


In [29]:
readInspect(r['result'])

,<CurrentPoolBalance>,<LiqBalance:<warehouse>>,<LiqCredit:<warehouse>>
Date,,,
2021-03-01,0.00,300.00,2000.00
2021-03-31,0.00,300.00,2000.00
2021-04-30,0.00,300.00,2000.00
2021-05-31,0.00,300.00,2000.00
2021-06-30,0.00,300.00,2000.00
2021-07-31,0.00,300.00,1998.99
2021-08-31,0.00,300.00,1997.36
2021-09-30,0.00,300.00,1994.96
2021-10-31,0.00,300.00,1991.81


In [30]:
r['result']['status']

,Date,From,To
0,2022-10-20,Warehousing,Called
1,2022-10-20,DealEnd,Clean Up
